Gradient Boosting Steps/Processes:
1. Initial prediction (Mean of the target)
2. Residual = Target - Prediction
    trees= []
3. For loop (loop for 1000's)
  4. Create a decision trees =>  tarin residual
  5. train(x, residuals)
  6. predicitons = predict(x) => predict residual
  7. Residual = Residual - learning_rate * prediction
  8. trees.append(tree)


1.Initial prediction => mean of target
2. REsidual = target- prediction
3. train decision tree = train residual
4. prediciton => prediciton of residual
5. new prediciton  = old prediciton + LR * prediciton
6.step 2 for all thre trees

In [9]:
import numpy as np

In [10]:
# Class Node (Root node, internal/decision node and leaf node)
class Node:
  def __init__(self, feature = None, threshold = None, left = None, right = None, value = None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

  def is_leaf_node(self):
    return self.value is not None

In [11]:
# Create a class DecisionTree that contains all methods used for prediciton
class DecisionTree:

    def __init__(self, max_depth = 3):
        self.max_depth = max_depth
        self.root = None

    def fit(self, x, y):
        self.root = self.build_tree(x, y)

    def mean_of_label(self, y): #change for boosting
        return np.mean(y)

    def split(self, x_column, split_threshold):
        left = np.argwhere(x_column <= split_threshold).flatten()
        right = np.argwhere(x_column > split_threshold).flatten()
        return left, right

    def entropy(self, y): #change for boosting
        return np.mean((y - np.mean(y)) ** 2)

    def information_gain(self, x_column, y, threshold):
        left, right = self.split(x_column, threshold)
        if len(left) == 0 or len(right) == 0:
            return 0
        else:
            p_left = len(left) / len(y)
            p_right = len(right) / len(y)
            weighted_entropy = p_left * self.entropy(y[left]) + p_right * self.entropy(
                y[right]
            )
            return weighted_entropy

    def best_split(self, x, y):
        num_sample, num_feature = x.shape
        best_gain = float("inf")
        split_feature = None
        split_threshold = None

        for feature in range(num_feature):
            x_column = x[:, feature]
            x_column_sorted = np.sort(x_column)
            threashold = (x_column_sorted[:-1] + x_column_sorted[1:]) / 2
            for th in threashold:
                weighted_entropy = self.information_gain(x_column, y, th)
                # print(f"Threshold {th} : {weighted_entropy}")

                if weighted_entropy < best_gain:
                    best_gain = weighted_entropy
                    split_feature = feature
                    split_threshold = th

        return split_feature, split_threshold

    def build_tree(self, x, y, depth = 0):
        n_lables = len(np.unique(y))
        if n_lables == 1 or depth >= self.max_depth:
            leaf_value = self.mean_of_label(y)
            return Node(value=leaf_value)

        best_feature, best_threshold = self.best_split(x, y)
        left_idxs, right_idxs = self.split(x[:, best_feature], best_threshold)
        left = self.build_tree(x[left_idxs, :], y[left_idxs], depth + 1)
        right = self.build_tree(x[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feature, best_threshold, left, right)

    def traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self.traverse_tree(x, node.left)
        else:
            return self.traverse_tree(x, node.right)

    def predict(self, x):
        predictions = np.array([self.traverse_tree(x, self.root) for x in x])
        return predictions

In [12]:
#Create a GradientBoosting class along with methods used to predict
class GradientBoosting:
  def __init__(self, n_estimator):
    self.n_estimator = n_estimator
    self.trees = []
    self.learning_rate = 0.1

  def fit(self, x, y):
    self.initial_prediction = np.mean(y)
    y_pred = np.full(y.shape, self.initial_prediction)
    for i in range(self.n_estimator):
      residual = y - y_pred
      tree = DecisionTree()
      tree.fit(x, residual)
      predicted_residual = tree.predict(x)
      y_pred = y_pred + self.learning_rate * predicted_residual
      self.trees.append(tree)

  def predict(self, x):
    y_pred = np.full(x.shape[0], self.initial_prediction )
    for i in range(self.n_estimator):
      tree = self.trees[i]
      prediction = tree.predict(x)
      y_pred = y_pred + self.learning_rate * prediction
    return y_pred

In [13]:
# Create a dummy dataset
x = np.array([[1,2], [2,3], [3,4], [4,5], [5,6]])
print(x)
y = np.array([1,1, 0, 0, 0])
print(y)

[[1 2]
 [2 3]
 [3 4]
 [4 5]
 [5 6]]
[1 1 0 0 0]


In [14]:
#Train the model
model = GradientBoosting(n_estimator=100)
model.fit(x, y)

In [17]:
#Test the model
y_predic = model.predict(x)


In [18]:
print(y)
print(y_predic)

[1 1 0 0 0]
[9.99984063e-01 9.99984063e-01 1.06245596e-05 1.06245596e-05
 1.06245596e-05]


In [19]:
#Create a r2 score method to check the result
def r2_score(y, y_predic):
    rss = np.sum((y - y_predic) ** 2 )
    tss = np.sum((y - np.mean(y)) ** 2)
    r2 = 1 - (rss/tss)
    return r2 * 100


In [23]:
#Print the result
print(r2_score(y, y_predic))

99.99999992944922
